# Milestone 1 – User Authentication System  
## PolicyNav – Public Policy Navigation Using AI  

This notebook demonstrates:

- Streamlit authentication UI  
- MySQL database integration  
- JWT-based authentication  
- Ngrok public deployment  

Developed for **Infosys Springboard Internship – Milestone-1**.


In [18]:
!pip install streamlit pyngrok PyJWT --quiet

In [19]:
import sqlite3

conn = sqlite3.connect("policynav.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL,
    security_question TEXT NOT NULL,
    security_answer TEXT NOT NULL
)
""")

conn.commit()
conn.close()

print("✅ SQLite database created")


✅ SQLite database created


In [20]:
%%writefile auth.py
import jwt
import datetime

SECRET_KEY = "policynav_secret_key"

def create_token(email):
    payload = {
        "email": email,
        "exp": datetime.datetime.utcnow() + datetime.timedelta(hours=2)
    }
    return jwt.encode(payload, SECRET_KEY, algorithm="HS256")


Overwriting auth.py


In [26]:
%%writefile app.py
import streamlit as st
import re
import sqlite3
from auth import create_token

# Page Config
st.set_page_config(page_title="PolicyNav | Secure Portal", layout="centered", page_icon="🔐")

# ---------- CSS (UNCHANGED) ----------
st.markdown("""
<style>
@import url('https://fonts.googleapis.com/css2?family=Plus+Jakarta+Sans:wght@400;600;700&display=swap');

.stApp {
    background: radial-gradient(circle at top right, #f8fafc, #e2e8f0);
    font-family: 'Plus Jakarta Sans', sans-serif;
}

.main-card {
    background: rgba(255, 255, 255, 0.95);
    padding: 3rem;
    border-radius: 24px;
    box-shadow: 0 20px 50px rgba(0, 0, 0, 0.05);
    border: 1px solid rgba(255, 255, 255, 0.3);
    margin-top: 20px;
}

.title-text {
    font-size: 2.5rem;
    font-weight: 800;
    text-align: center;
    color: #1e293b;
    letter-spacing: -1px;
    margin-bottom: 0.5rem;
}

.subtitle-text {
    text-align: center;
    color: #64748b;
    font-size: 1rem;
    margin-bottom: 2rem;
}

.stButton > button {
    width: 100%;
    border-radius: 12px;
    padding: 0.6rem 1rem;
    background-color: #2563eb;
    color: white;
    border: none;
    font-weight: 600;
}

.stButton > button:hover {
    background-color: #1d4ed8;
    box-shadow: 0 10px 15px -3px rgba(37, 99, 235, 0.3);
    transform: translateY(-2px);
}

.stTextInput input {
    border-radius: 10px;
    border: 1px solid #e2e8f0;
    padding: 0.5rem;
}

#MainMenu {visibility: hidden;}
footer {visibility: hidden;}
section[data-testid="stSidebar"] {display: none;}
</style>
""", unsafe_allow_html=True)


# ---------- DATABASE ----------
def get_connection():
    return sqlite3.connect("policynav.db", check_same_thread=False)


def switch(page):
    st.session_state.page = page
    st.rerun()


# ---------- SESSION ----------
for key, default in [("logged_in", False), ("username", ""), ("token", None), ("page", "Login")]:
    if key not in st.session_state:
        st.session_state[key] = default


# ---------- UI WRAPPER ----------
st.markdown('<div class="main-card">', unsafe_allow_html=True)
st.markdown('<div class="title-text">PolicyNav</div>', unsafe_allow_html=True)
st.markdown('<div class="subtitle-text">Next-gen policy intelligence portal</div>', unsafe_allow_html=True)


# ================= SIGNUP =================
if st.session_state.page == "Signup":

    st.subheader("Create your account")

    col1, col2 = st.columns(2)
    with col1:
        username = st.text_input("Full Name")
        email = st.text_input("Work Email")
    with col2:
        password = st.text_input("Password", type="password")
        confirm = st.text_input("Confirm Password", type="password")

    question = st.selectbox("Security Question", [
        "What was the name of your first pet?",
        "In what city were you born?",
        "What is your mother's maiden name?"
    ])
    answer = st.text_input("Security Answer", type="password")

    if st.button("Register Now"):

        # ---------- VALIDATION ----------
        if not all([username, email, password, confirm, answer]):
            st.error("All fields are mandatory.")

        elif not re.match(r"[^@]+@[^@]+\.[a-zA-Z]{2,}", email):
            st.error("Enter a valid email address.")

        elif not password.isalnum():
            st.error("Password must be alphanumeric.")

        elif password != confirm:
            st.error("Passwords do not match.")

        else:
            conn = get_connection()
            cur = conn.cursor()
            try:
                cur.execute(
                    "INSERT INTO users (username,email,password,security_question,security_answer) VALUES (?,?,?,?,?)",
                    (username, email, password, question, answer)
                )
                conn.commit()

                # 🔐 JWT issued on signup
                st.session_state.token = create_token(email)

                st.success("Registration successful. Please sign in.")
                switch("Login")

            except sqlite3.IntegrityError:
                st.error("Email already registered.")
            finally:
                conn.close()

    if st.button("Already have an account? Login"):
        switch("Login")


# ================= LOGIN =================
elif st.session_state.page == "Login" and not st.session_state.logged_in:

    email = st.text_input("Email Address")
    password = st.text_input("Password", type="password")

    if st.button("Sign In"):

        if not email or not password:
            st.error("Both fields are required.")

        else:
            conn = get_connection()
            cur = conn.cursor()
            cur.execute("SELECT username,password FROM users WHERE email=?", (email,))
            user = cur.fetchone()
            conn.close()

            if user and user[1] == password:
                st.session_state.logged_in = True
                st.session_state.username = user[0]

                # 🔐 JWT attached to session
                st.session_state.token = create_token(email)

                st.success("Authentication successful.")
                st.rerun()
            else:
                st.error("Invalid email or password.")

    st.divider()
    c1, c2 = st.columns(2)
    with c1:
        if st.button("New Account"):
            switch("Signup")
    with c2:
        if st.button("Forgot Password?"):
            switch("Forgot")


# ================= FORGOT PASSWORD =================
elif st.session_state.page == "Forgot":

    st.subheader("Reset Account Access")
    email = st.text_input("Registered Email")

    if st.button("Find Account"):

        conn = get_connection()
        cur = conn.cursor()
        cur.execute("SELECT security_question,security_answer FROM users WHERE email=?", (email,))
        data = cur.fetchone()
        conn.close()

        if data:
            st.session_state.reset_email = email
            st.session_state.q_text = data[0]
            st.session_state.a_val = data[1]
        else:
            st.error("No account found with this email.")

    if "q_text" in st.session_state:
        st.write(f"**Security Question:** {st.session_state.q_text}")
        ans = st.text_input("Answer", type="password")
        new_p = st.text_input("New Password", type="password")

        if st.button("Verify & Reset"):

            if not new_p.isalnum():
                st.error("Password must be alphanumeric.")

            elif ans == st.session_state.a_val:

                conn = get_connection()
                cur = conn.cursor()
                cur.execute("UPDATE users SET password=? WHERE email=?", (new_p, st.session_state.reset_email))
                conn.commit()
                conn.close()

                # 🔐 New JWT issued
                st.session_state.token = create_token(st.session_state.reset_email)

                st.success("Password reset successful. Please login again.")
                st.session_state.clear()
                switch("Login")
            else:
                st.error("Incorrect security answer.")

    if st.button("Back to Login"):
        switch("Login")


# ================= DASHBOARD =================
elif st.session_state.logged_in and st.session_state.token:

    st.markdown(f"### Welcome, **{st.session_state.username}**")
    st.info("Secure session established via JWT authentication.")

    st.write("---")
    st.write("Protected PolicyNav dashboard content will appear here.")

    if st.button("Secure Logout"):
        st.session_state.clear()
        switch("Login")


st.markdown('</div>', unsafe_allow_html=True)


Overwriting app.py


In [22]:
!ngrok config add-authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [23]:
from pyngrok import ngrok
import subprocess, time

# kill anything old
ngrok.kill()
!pkill -f streamlit
!pkill ngrok

# start streamlit
process = subprocess.Popen(["streamlit", "run", "app.py"])

# wait for server
time.sleep(10)

# open new tunnel
public_url = ngrok.connect(8501)
print("🌍 OPEN THIS LINK:", public_url)


🌍 OPEN THIS LINK: NgrokTunnel: "https://justine-strikebound-independently.ngrok-free.dev" -> "http://localhost:8501"
